In [23]:
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sbn
import scipy.stats as stats

We will add all the additional features to our dataframe

In [4]:
train = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
transactions = pd.read_csv('transactions.csv')
holidays = pd.read_csv('holidays_events.csv')

In [12]:
#this is Lars's code to add in a time variable
dates_dt = pd.to_datetime(train['date'])
dates_dt_min = dates_dt.min()
dates_days = (dates_dt - dates_dt_min).dt.days

display(dates_days)
train['time'] = dates_days
display(train)

#we also add a time variable to the holidays dataframe
holidays_days = (pd.to_datetime(holidays['date']) - dates_dt_min).dt.days

display(holidays_days)
holidays['time'] = holidays_days
display(holidays)

0             0
1             0
2             0
3             0
4             0
           ... 
3000883    1687
3000884    1687
3000885    1687
3000886    1687
3000887    1687
Name: date, Length: 3000888, dtype: int64

,id,date,store_nbr,family,sales,onpromotion,time
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0
1,1,2013-01-01,1,BABY CARE,0.000,0,0
2,2,2013-01-01,1,BEAUTY,0.000,0,0
3,3,2013-01-01,1,BEVERAGES,0.000,0,0
4,4,2013-01-01,1,BOOKS,0.000,0,0
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687


0      -305
1      -275
2      -264
3      -262
4      -255
       ... 
345    1816
346    1817
347    1818
348    1819
349    1820
Name: date, Length: 350, dtype: int64

,date,type,locale,locale_name,description,transferred,time
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,-305
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,-275
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,-264
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,-262
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,-255
...,...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,1816
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,1817
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,1818
348,2017-12-25,Holiday,National,Ecuador,Navidad,False,1819


First we define a function to check if a day is a holiday or not for a specific store, dependent on the region and locale of the store.

In [13]:
#write a function to check if a day is a holiday or event for a specific store
def is_holiday(t, st):
    #check if the day shows up in the holidays_events dataset
    if np.isin(t,holidays['time']):
        #if it is record the index of the holiday
        holidayindex = np.where(holidays['time'] == t)[0][0]
        storeindex = np.where(stores['store_nbr'] == st)[0][0]
        #check if the holiday is transferred away from this day
        if holidays['transferred'][holidayindex]:
            #if the holiday is transferred treat it like a normal day
            return 'No'
        else:
            #check if the holiday is transferred from another holiday
            if holidays['type'][holidayindex] == 'Transfer':
                #if it is transferred treat it like a holiday
                d = 'Holiday'
            else:
                d = holidays['type'][holidayindex]
            #check if the holiday affects the region
            if holidays['locale'][holidayindex] == 'National':
                return d
            else:
                #check if the holiday is regional or local
                if holidays['locale'][holidayindex] == "Regional":
                    #check if the store is in the region
                    if stores['state'][storeindex] == holidays['locale_name'][holidayindex]:
                        return d
                    else:
                        return 'No'
                else:
                    #check if the store is in the locality of the local holiday
                        if stores['city'][storeindex] == holidays['locale_name'][holidayindex]:
                            return d
                        else:
                            return 'No'
    #if it is not a holiday return 'No'
    else:
        return 'No'

                    

We then test that it works

In [14]:
print(is_holiday(t=341, st = 38))
print(is_holiday(t=341, st = 3))
print(is_holiday(t=0, st = 3))

Holiday
No
Holiday


Now we add holidays to the test dataframe

In [15]:
vector_holiday = np.vectorize(is_holiday)

holiday_ar = vector_holiday(train['time'],train['store_nbr'])

display(holiday_ar)

array(['Holiday', 'Holiday', 'Holiday', ..., 'No', 'No', 'No'],
      dtype='<U10')

In [16]:
train['holiday'] = holiday_ar
display(train)

,id,date,store_nbr,family,sales,onpromotion,time,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0,Holiday
1,1,2013-01-01,1,BABY CARE,0.000,0,0,Holiday
2,2,2013-01-01,1,BEAUTY,0.000,0,0,Holiday
3,3,2013-01-01,1,BEVERAGES,0.000,0,0,Holiday
4,4,2013-01-01,1,BOOKS,0.000,0,0,Holiday
...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687,No
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687,No
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687,No
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687,No


Now we add the rest of our features

In [17]:
#This is Ciaran's code to add days since last paycheck for public employees

def day_date_int(str):
    return 10 * int(str[-2]) + int(str[-1])


lastdayofmonth = {'01-31', '02-28', '03-31', '04-30', '05-31', '06-30', '07-31', '08-31', '09-30', '10-31', '11-30',
                  '12-31'}


def datetodayssincelastpayday(str):
    if str == '2016-02-28':
        return 13
    if str == '2016-02-29':
        return 0
    if str[-5:] in lastdayofmonth:
        return 0
    else:
        if day_date_int(str) >= 15:
            return day_date_int(str) - 15
        else:
            return day_date_int(str)


arr = [0] * len(train['date'])
for i in range(len(train['date'])):
    arr[i] = datetodayssincelastpayday(train['date'][i])
train['dayssincepaid'] = arr

In [18]:
display(train)

,id,date,store_nbr,family,sales,onpromotion,time,holiday,dayssincepaid
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0,Holiday,1
1,1,2013-01-01,1,BABY CARE,0.000,0,0,Holiday,1
2,2,2013-01-01,1,BEAUTY,0.000,0,0,Holiday,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,0,Holiday,1
4,4,2013-01-01,1,BOOKS,0.000,0,0,Holiday,1
...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687,No,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687,No,0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687,No,0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687,No,0


Now we add Lars's code to complete the oil data to fill in the gaps on weekends when oil was not reported

In [19]:
oil_txf = oil.copy()
oil_txf['date'] = pd.to_datetime(oil['date'])
oil_txf['day_num'] = (oil_txf['date'] - dates_dt_min).dt.days

In [20]:
# def get_oil_price(date):
#     if date.day_of_week == 5:
#         return oil_txf
# date = oil_filled['date'].iloc[1]
def get_oil_price(date : dt.datetime):
    # print(date)
    # print(date.day_of_week)
    tgt = date
    if date.day_of_week == 5:
        tgt = date - dt.timedelta(days = 1)
    if date.day_of_week == 6:
        tgt = date - dt.timedelta(days = 2)
    # print(tgt)
    price = oil_txf[oil_txf['date'] == tgt]['dcoilwtico'].iloc[0]
    return price

In [21]:
time_arange = np.arange(1688)
oil_filled = pd.DataFrame(index = time_arange, data = {
    'day_num' : pd.to_timedelta(time_arange, unit = 'days')
})
oil_filled['date'] = oil_filled['day_num'] + dates_dt_min
oil_filled['price'] = oil_filled['date'].apply(get_oil_price)
display(oil_filled)

,day_num,date,price
0,0 days,2013-01-01,NaN
1,1 days,2013-01-02,93.14
2,2 days,2013-01-03,92.97
3,3 days,2013-01-04,93.12
4,4 days,2013-01-05,93.12
...,...,...,...
1683,1683 days,2017-08-11,48.81
1684,1684 days,2017-08-12,48.81
1685,1685 days,2017-08-13,48.81
1686,1686 days,2017-08-14,47.59


Now we add the filled in oil feature to our dataframe

In [22]:
train['oil'] = train['time'].apply(oil_filled['price'].get)

In [24]:
display(train)

,id,date,store_nbr,family,sales,onpromotion,time,holiday,dayssincepaid,oil
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0,Holiday,1,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,0,Holiday,1,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,0,Holiday,1,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,0,Holiday,1,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,0,Holiday,1,NaN
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687,No,0,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687,No,0,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687,No,0,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687,No,0,47.57


In [25]:
train['dayofweek'] = pd.to_datetime(train['date']).dt.day_of_week
train['month'] = pd.to_datetime(train['date']).dt.month

In [26]:
display(train)

,id,date,store_nbr,family,sales,onpromotion,time,holiday,dayssincepaid,oil,dayofweek,month
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0,Holiday,1,NaN,1,1
1,1,2013-01-01,1,BABY CARE,0.000,0,0,Holiday,1,NaN,1,1
2,2,2013-01-01,1,BEAUTY,0.000,0,0,Holiday,1,NaN,1,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,0,Holiday,1,NaN,1,1
4,4,2013-01-01,1,BOOKS,0.000,0,0,Holiday,1,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687,No,0,47.57,1,8
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687,No,0,47.57,1,8
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687,No,0,47.57,1,8
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687,No,0,47.57,1,8
